## Kavir_RAG
Retrieval-Augmented Generation
<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
Retrieval = بازیابی (درست مثل سرچ کردن و پیدا کردن یک متن مرتبط از میان کلی سند)
Augmented = تقویت‌شده / غنی‌شده (یعنی اطلاعات جدیدی به چیزی اضافه شده)
Generation = تولید متن (کاری که مدل زبانی انجام می‌ده: جمله‌سازی، پاسخ‌دهی)
</div>

<div style="direction: rtl;">


# تعریف پروژه (گام ۰: صورت‌مسئله و برنامهٔ کار)

## نام‌ها

* **نام پروژه (repo):** `Kavir-RAG`  *(کویر: محلی، مستقل، خلوت)*
* **نام نوت‌بوک اصلی:** `Kavir_RAG_MVP.ipynb`

## هدف فنی

یک **سرور RAG کاملاً لوکال و آفلاین** برای تحلیل اسناد چندفرمت (PDF, DOCX, XLSX/CSV, TXT, code snippets, IPYNB) با:

* ایندکس‌سازی (FAISS/Chroma) + امبدینگ چندزبانه (fa/en)
* بازیابی معنایی + (اختیاری) ریرنکر
* پاسخ LLM لوکال با **ارجاع به منبع**
* API مبتنی بر **FastAPI** (endpoints: `/upload`, `/query`, `/chat`, `/reindex`) با **توکن ساده**
* اجرا روی i5 / 8GB RAM (CPU-only؛ 16GB روان‌تر)

## پیکرهٔ فنی پیشنهادی

* **LLM**: Qwen2-7B-Instruct (Q4\_0 via Ollama/llama.cpp) – کم‌مصرف و چندزبانه
* **Embedding**: `intfloat/multilingual-e5-small` (CPU-friendly)
* **Vector DB**: FAISS (IndexFlatIP + نُرمال‌سازی → cosine)
* **Re-rank (اختیاری)**: `bge-reranker-base` (CPU)
* **Parserها**: PyMuPDF (PDF)، python-docx، pandas (CSV/XLSX)، nbformat (ipynb)، Tesseract (OCR در صورت نیاز)
* **API**: FastAPI + Uvicorn
* **UI سبک (اختیاری)**: Streamlit (یا React ساده)
* **امنیت LAN**: توکن ساده + (اختیاری) Nginx/Caddy با TLS داخلی


## خروجی‌های نسخهٔ پایه (MVP)

* Endpointهای `/upload`, `/query`, `/reindex` (و `/chat` سبک)
* بازیابی top-k با citation
* ذخیرهٔ ایندکس روی دیسک و بارگذاری مجدد
* اجرای کامل روی یک ماشین بدون اینترنت (بعد از دانلود اولیهٔ مدل‌ها)

## زمان‌بندی و هزینه (الگوی آپ‌ورک)

* **MVP (فقط API+RAG)**: \~ **۱ روز کاری**
  شکست به تسک‌ها: تعاریف و اسکلت (۱س) • نصب/مدل (۲س) • ingest+index (۲–۳س) • RAG+API (۲س) • تست سریع (۱س).
* **کامل‌تر (UI، امنیت LAN، بکاپ، ریرنکر، هیبرید سرچ)**: **۲–۴ روز کاری**
* **تخمین هزینه** (فرضِ فریلنسری استاندارد):

  * MVP: **۲۰۰–۳۰۰ دلار**
  * کامل‌تر: **۵۰۰–۸۰۰ دلار**
* **آپشن توسعهٔ حرفه‌ای (۳–۷ روز اضافه، +۷۰۰ تا +۱۵۰۰ دلار بسته به دامنه):**

  * هیبرید سرچ (BM25 + dense) و تیونینگ ریتریور
  * ریرنکر پیش‌فرض + A/B ارزیابی پاسخ‌ها
  * Session memory برای چت چندمرحله‌ای
  * Docker Compose + Nginx/Caddy + TLS داخلی
  * Role-based tokens، لاگ و Auditing
  * Sandbox خواندن امنِ ipynb (بدون اجرای کُد کاربر)
  * تست‌های واحد/تکاملی + بنچمارک کوچک کیفیت RAG
  * اسکریپت پشتیبان‌گیری ایندکس + بازیابی

## ریسک‌ها و قیود

* 8GB RAM کفِ قابل اجراست؛ زمان ساخت امبدینگ اولیه کندتر است.
* فایل‌های تصویریِ فرمول نیازمند OCR هستند و ممکن است کیفیت استخراج متغیر باشد.
* بارِ نخست نیاز به دانلود مدل‌ها دارد؛ سپس کاملاً آفلاین.

---

# گام بعدی (مرحله ۱: اسکلت پروژه و آماده‌سازی محیط)

در این مرحله فقط **ساختار پوشه‌ها** و یک **venv سبک** ایجاد می‌کنیم و فهرست **dependencies** را می‌نویسیم (بدون نصب سنگین مدل‌ها هنوز).
طبق «پرامپتِ قدم»، **هیچ کُدی** تا وقتی تأیید ندهی نمایش نمی‌دهم.


</div>

In [11]:
'''Kavir-RAG/
  data/
    raw/           # raw uploaded files
    ocr/           # OCR output (if needed)
    index/         # FAISS index stored on disk
  models/          # cached models (optional)
  app/
    api.py         # FastAPI endpoints
    rag_core.py    # retrieval + prompt composition + generation
    ingest.py      # file parsers + chunking + embeddings
    auth.py        # simple token-based authentication
    utils.py       # helper functions
  notebooks/
    Kavir_RAG_MVP.ipynb
  configs/
    settings.yaml  # configuration file
  tests/           # unit/integration tests
  README.md        # project documentation
  '''


'Kavir-RAG/\n  data/\n    raw/           # raw uploaded files\n    ocr/           # OCR output (if needed)\n    index/         # FAISS index stored on disk\n  models/          # cached models (optional)\n  app/\n    api.py         # FastAPI endpoints\n    rag_core.py    # retrieval + prompt composition + generation\n    ingest.py      # file parsers + chunking + embeddings\n    auth.py        # simple token-based authentication\n    utils.py       # helper functions\n  notebooks/\n    Kavir_RAG_MVP.ipynb\n  configs/\n    settings.yaml  # configuration file\n  tests/           # unit/integration tests\n  README.md        # project documentation\n  '

In [12]:
!touch Dockerfile
!touch requirements.txt
!mkdir -p src
!touch src/main.py

In [13]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">

### 2. FAISS دقیقاً چیکار می‌کنه؟

* **FAISS (Facebook AI Similarity Search)** یه کتابخونه است برای **جستجوی سریع در وکتورها**.
* وقتی ما متن‌ها رو به امبدینگ (بردار) تبدیل کنیم، FAISS کمک می‌کنه سریع‌ترین متن‌های مشابه رو پیدا کنیم.
* خلاصه: مغز قسمت «Retrieve» در RAG هست.

---

### 3. الان کجاییم؟

ما Step 1 رو اینجوری پیش بردیم:

* ✅ ساخت پوشه‌ها و فایل‌ها
* ✅ نوشتن و تست `main.py` (FastAPI)
* ✅ ساخت و نصب `requirements.txt` (بدون faiss روی مک)

🔜 مرحله بعدی همون Step 1:

* نوشتن **Dockerfile** → تا محیط لینوکسی بسازیم و اونجا `faiss-cpu` هم درست نصب بشه.


</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
# مرحله ۱: آماده‌سازی محیط و اسکلت پروژه (Kavir-RAG)

## هدف
ایجاد اسکلت اولیه پروژه با FastAPI و Docker برای اجرای سرور محلی.

## اقدامات انجام‌شده
- ساخت پوشه‌ها و فایل‌های پایه (`src/main.py`, `requirements.txt`, `Dockerfile`)
- نوشتن کد تستی FastAPI با endpoint `/ping`
- نصب پکیج‌های پایه با `requirements.txt` (بدون faiss روی مک)
- نوشتن Dockerfile و Build ایمیج با `python:3.11-slim`
- اجرای کانتینر و تست `/ping` و `/docs` از داخل Docker
- افزودن `.gitignore` برای جلوگیری از push فایل‌های حجیم (مدل‌ها، دیتا، ایندکس)

## وضعیت فعلی
✅ سرور FastAPI داخل کانتینر Docker به‌درستی اجرا شد و تست اولیه موفق بود.

## گام بعدی (مرحله ۲)
- ماژول ingestion برای خواندن فایل‌های PDF, Word, CSV, Notebook
- chunking متن
- تولید امبدینگ (multilingual-e5-small)
- ذخیره در ایندکس FAISS

</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
سلول 1 — ساخت پوشه‌ها و فایل‌های نمونه (صرفاً پایتون)
</div>

In [14]:
import os, json, textwrap, pathlib

# ریشه پروژه را همین پوشه نوت‌بوک فرض می‌کنیم
BASE = pathlib.Path(".").resolve()
(DATA_RAW, DATA_CHUNKS, SRC) = (BASE/"data"/"raw", BASE/"data"/"chunks", BASE/"src")
for p in (DATA_RAW, DATA_CHUNKS, SRC):
    p.mkdir(parents=True, exist_ok=True)

# نمونه فایل متنی (اعداد انگلیسی تا مشکلی پیش نیاد)
sample_txt = """این یک متن آزمایشی برای تست چانکینگ است.
هدف: تقسیم متن به قطعات 800 کاراکتری با همپوشانی 100.
"""
(DATA_RAW/"sample.txt").write_text(sample_txt, encoding="utf-8")

# نمونه CSV
csv_txt = "region,price,year,manufacturer,model\nseattle,12000,2012,toyota,corolla\nla,8000,2010,honda,civic\n"
(DATA_RAW/"cars.csv").write_text(csv_txt, encoding="utf-8")

print("ساخته شد:", DATA_RAW)
print([p.name for p in DATA_RAW.iterdir()])


ساخته شد: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw
['cars.csv', 'sample.txt']


<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
سلول 2 — اجرای ingestion.py از داخل نوت‌بوک با همان Python فعلی
</div>

In [16]:
import sys, subprocess, pathlib
REQ = pathlib.Path("requirements.txt").resolve()
print("Using interpreter:", sys.executable)
print("Installing from:", REQ)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(REQ)])


Using interpreter: /usr/local/bin/python3
Installing from: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/requirements.txt
  Using cached pymupdf-1.26.4-cp39-abi3-macosx_10_9_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.4-cp39-abi3-macosx_10_9_x86_64.whl (23.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdf2image]/3 [pymupdf]


0

In [17]:
import sys, subprocess, shlex, pathlib

PY = shlex.quote(sys.executable)  # مسير همين پایتون (مثلاً /usr/bin/python3.11)
cmd = (
    f'{PY} src/ingestion.py '
    f'--path {shlex.quote(str(DATA_RAW))} '
    f'--out {shlex.quote(str(DATA_CHUNKS))} '
    f'--chunk-size 800 --overlap 100 --preview'
)
print("RUN:", cmd)

proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print("STDOUT:\n", proc.stdout)
print("STDERR:\n", proc.stderr)
if proc.returncode != 0:
    raise RuntimeError(f"ingestion.py failed with code {proc.returncode}")


RUN: /usr/local/bin/python3 src/ingestion.py --path '/Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw' --out '/Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/chunks' --chunk-size 800 --overlap 100 --preview
STDOUT:
 ✅ Ingested: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw/cars.csv
   chars=96  chunks=1  -> /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/chunks/cars__030047719b5e.jsonl
   preview: region,price,year,manufacturer,model seattle,12000,2012,toyota,corolla la,8000,2010,honda,civic  ...
✅ Ingested: /Users/macbookpro/AMIR_DATA/00_Project/VS

سلول 3 — بررسی خروجی‌ها (خواندن JSONL با پایتون)

In [18]:
from glob import glob
import json, pathlib
CHUNKS_DIR = pathlib.Path("data/chunks")

files = sorted(glob(str(CHUNKS_DIR/"*.jsonl")))
print("Found:", len(files), "chunk files")
for fp in files:
    print("—", pathlib.Path(fp).name)

if not files:
    raise SystemExit("هیچ فایل چانکی پیدا نشد. اول ingestion رو اجرا کن.")

total_lines = 0
for fp in files:
    print("\n== Preview:", pathlib.Path(fp).name, "==")
    with open(fp, "r", encoding="utf-8") as f:
        # سه خط اول برای نمونه
        for i in range(3):
            line = f.readline()
            if not line: break
            obj = json.loads(line)
            total_lines += 1
            print(f"[{i+1}] chunk_index={obj['chunk_index']}  range={obj['char_range']}  src={obj['metadata']['source_name']}")
            print("   text:", obj["text"][:120].replace("\n", " "), "...")
print("\nTotal previewed lines:", total_lines)


Found: 2 chunk files
— cars__030047719b5e.jsonl
— sample__54e5a362dddd.jsonl

== Preview: cars__030047719b5e.jsonl ==
[1] chunk_index=0  range=[0, 96]  src=cars.csv
   text: region,price,year,manufacturer,model seattle,12000,2012,toyota,corolla la,8000,2010,honda,civic  ...

== Preview: sample__54e5a362dddd.jsonl ==
[1] chunk_index=0  range=[0, 95]  src=sample.txt
   text: این یک متن آزمایشی برای تست چانکینگ است. هدف: تقسیم متن به قطعات 800 کاراکتری با همپوشانی 100.  ...

Total previewed lines: 2


<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
باشه امیر 👌 یک مرور جامع از پروژه RAG لوکال تا اینجا (طبق تعریف اولیه‌ای که گذاشتی: ingest → embed → index → retrieval → API)

---

# 📝 خلاصه پیشرفت پروژه RAG (تا اینجا)

## ✅ کارهای انجام‌شده

### 1. ساختار و محیط

* پوشه‌بندی پروژه (`src/`, `data/`, `requirements.txt`, `Dockerfile`) ساخته شد.
* `.gitignore` تنظیم شد → دیتای سنگین (`data/`, `.cache/`) و کش HuggingFace نادیده گرفته می‌شوند.
* Dockerfile نهایی شد:

  * بیس: `python:3.11-slim`
  * نصب پکیج‌های سیستمی (Tesseract, Poppler)
  * نصب پکیج‌های پایتون (FastAPI, torch (CPU), faiss-cpu, sentence-transformers, pandas, …)
  * اضافه شدن `ENV HF_HOME=/root/.cache/huggingface` برای مدیریت کش
  * CMD → اجرای `uvicorn` با یک ورکر

### 2. تست محیط و کتابخانه‌ها

* اجرای کانتینر و تست نصب:

  * Torch (CPU-only) ✔
  * FAISS ✔
  * Sentence-Transformers ✔
* مدل `intfloat/multilingual-e5-small` یک‌بار لود شد، کش پایدار روی میزبان تنظیم شد (`.cache/hf`).

### 3. Ingestion + Chunking

* `src/ingestion.py` نوشته شد:

  * پشتیبانی از PDF, DOCX, TXT, CSV
  * چانکینگ متنی (JSONL خروجی → `data/chunks/`)
* تست شد (روی ۲ فایل نمونه) → خروجی چانک OK.

### 4. Embedding + Indexing

* `src/embeddings.py` نوشته شد:

  * چانک‌ها خوانده شد.
  * امبدینگ‌ها با `multilingual-e5-small` ساخته شد.
  * L2 normalize + ذخیره در FAISS (IndexFlatIP)
  * ذخیره متادیتا (`meta.jsonl`) و ایندکس (`faiss.index`) در `data/index/`.
* تست اجرا شد → `Index size: 2 vectors of dim 384`.

### 5. Retrieval

* `src/search.py` نوشته شد:

  * کوئری کاربر امبد شد.
  * FAISS جست‌وجو کرد.
  * متادیتا و متن چانک‌های برتر چاپ شد.
* تست روی کوئری → نتایج درست برگشت.

### 6. API (FastAPI)

* `src/main.py` به‌روزرسانی شد:

  * `/ping` (health check)
  * `/query` (retrieval endpoint)
* تست Swagger (`/docs`) انجام شد → کوئری و نتایج OK.

---

## 🟡 کارهایی که هنوز مانده

### مرحله‌های اصلی پروژه

1. **بهبود اسکیما چانک‌ها**

   * اضافه کردن فیلدهای `id`, `doc_id`, `chunk_idx`, `start`, `end` به خروجی ingestion.
   * اجرای دوباره ingestion + embeddings → ایندکس کامل‌تر.

2. **Re-chunk & Re-index**

   * بعد از اصلاح اسکیما، تمام فایل‌ها دوباره چانک شوند.
   * ایندکس جدید ساخته شود.

3. **بهبود API**

   * `/query`: خروجی تمیزتر (id, doc\_id, text\_preview, score).
   * اضافه کردن `/upload`: آپلود فایل (PDF, DOCX, …) و اجرای ingestion → آپدیت ایندکس.
   * اضافه کردن `/reindex`: ساخت دوباره ایندکس از صفر.
   * اختیاری: `/chat`: رپ کردن retrieval + LLM (Qwen2-7B-Instruct q4).

4. **اتصال LLM**

   * بعد از retrieval، کوئری + context به LLM داده شود.
   * مدل هدف: **Qwen2-7B-Instruct (q4)**.
   * اجرا روی CPU (i5 / 8GB) → نیاز به مدیریت حافظه و batch کوچک.

5. **UI اختیاری**

   * Streamlit یا React برای نمایش نتایج و تعامل ساده‌تر.

---

## 🔮 جمع‌بندی

* تا اینجا **کل مسیر ingest → embed → index → retrieval → API** برای MVP تست شد و جواب داده.
* پروژه الان یک **RAG پایه** داره که میشه رویش توسعه داد.
* کار اصلی مونده: **اضافه کردن LLM به retrieval (مرحله پاسخ‌دهی)** و **endpointهای تکمیلی** برای ingestion/reindex/upload.



</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
عالیه امیر 👌

📌 خلاصهٔ این گام:

* `embeddings.py` با موفقیت اجرا شد.
* از روی **۴ چانک** امبدینگ ساخته شد (هر بردار 384 بعدی).
* ایندکس FAISS (`faiss.index`) و متادیتا (`meta.jsonl`) ساخته شد.
* بکاپ‌های قدیمی هم نگه داشته شدند.
* هم داخل کانتینر و هم روی مک در مسیر `data/index/` قابل دیدن هستند. ✅

---

### وضعیت پروژه RAG تا اینجا

1. **Ingestion** → `chunks.jsonl` و `manifest.jsonl` → آماده
2. **Embedding + Indexing** → `faiss.index` و `meta.jsonl` → آماده

---

### 🟡 گام بعدی پیشنهادی

اضافه‌کردن endpoint `/query` در `search.py` یا `main.py` برای این کار:

* گرفتن query از کاربر
* ساخت embedding همان query
* سرچ در FAISS
* برگرداندن: `id`, `doc_id`, `chunk_idx`, `score`, `text_preview`

---

👉 می‌خوای مرحله بعد بریم سراغ ساخت `/query` endpoint؟

</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
باشه امیر 👌 این خلاصهٔ فشرده از مسیری که تا الان رفتیم:

---

## 📌 خلاصه مراحل انجام‌شده در پروژه RAG

### 1. پایه پروژه و Docker

* ساختار پوشه‌ها (`src/`, `data/`, `.cache/hf`) تنظیم شد.
* **Dockerfile** با ایمیج `python:3.11-slim` ساخته شد.
* Volumeها برای کد، دیتا و کش HuggingFace مونت شد.

### 2. API اولیه (FastAPI)

* `src/main.py` نوشته شد.
* `/ping` → تست سلامت.
* سرور با **uvicorn** در کانتینر بالا آمد.

### 3. Ingestion

* `src/ingestion.py` بازنویسی شد.
* **chunking** روی `demo.txt` تست شد.
* خروجی‌ها:

  * `data/chunks/chunks.jsonl`
  * `data/chunks/manifest.jsonl`

### 4. Embedding & Index

* `src/embeddings.py` ساخته شد.
* مدل **multilingual-e5-small** استفاده شد.
* خروجی:

  * `data/index/faiss.index`
  * `data/index/meta.jsonl`

### 5. جستجو

* `src/search.py` (CLI) → تست ok.
* `/query` در `main.py` → تست ok.

### 6. Reindex

* `/reindex` به `main.py` اضافه شد.
* اجرای pipeline embeddings → index + meta دوباره ساخته شد.
* همزمان index در حافظه reload شد.

### 7. Upload (txt)

* `/upload` اضافه شد.
* فایل txt ذخیره → chunk → append به `chunks.jsonl` و `manifest.jsonl`.
* قابلیت **reindex=true** برای ساخت index خودکار.
* تست عملی → جواب ok.

### 8. Dependencies دائمی

* `requirements.txt` و `Dockerfile` بازنویسی شدند.
* پکیج‌های لازم (`fastapi`, `uvicorn`, `sentence-transformers`, `faiss-cpu`, `pandas`, `pypdf`, `python-docx`, `pdf2image`, `pytesseract`, …) اضافه شدند.
* Build با کش بهینه انجام شد.

1 mehr

</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
* لپ‌تاپت: **MacBook Pro 2015 (Intel-based)**
* سبک پاسخ: کوتاه (مگر اینکه بخوای توضیح کامل).
* همیشه قبل از دادن کد جدید باید بپرسم **آیا فایل/کد فعلی درست اجرا شده یا نه**.
* هر بار باید مشخص کنم:

  1. فایل جدید جایگزین قبلی بشه یا با اسم جدید ذخیره بشه.
  2. کجا اجرا بشه (هاست یا کانتینر).
  3. چند ترمینال نیاز داره (مثلاً ترمینال دوم برای تست).
* پروژه‌ها باید **مرحله به مرحله** پیش برن (پرامپت قدم).
* پروژه RAG لوکال با **Qwen2-7B-Instruct (q4)** و **multilingual-e5-small**، اجرا روی CPU، Dockerized، بدون API خارجی.

---

# 🗂️ حافظه پروژه RAG (جزئیات کار)

1. **ساختار و زیرساخت**

   * دایرکتوری: `src/`, `data/`, `.cache/hf`
   * Dockerfile: Python 3.11 slim + deps (FastAPI, uvicorn, sentence-transformers, faiss-cpu, pandas, pypdf, python-docx, pdf2image, pytesseract, Pillow, tqdm, numpy, scikit-learn, torch, …)
   * Build با کش (BuildKit)

2. **اسکیما و کدها**

   * `schema_defs.py` تعریف شده (DocumentManifest, TextChunk)
   * `ingestion.py`: متن را chunk می‌کند → خروجی:

     * `data/chunks/chunks.jsonl`
     * `data/chunks/manifest.jsonl`
   * `embeddings.py`: chunks → embeddings (multilingual-e5-small) → L2 normalize → FAISS → `faiss.index` + `meta.jsonl`
   * `search.py`: CLI جستجو روی index
   * `main.py`: FastAPI app با مسیرها:

     * `/ping`
     * `/query` (Top-k search)
     * `/reindex` (بازسازی index با `embeddings.py`)
     * `/upload` (فقط txt → ذخیره در `data/raw`, append به chunks + manifest, optional reindex)

3. **ترتیب مراحل (پرامپت قدم)**

   * همیشه قبل از دادن کد جدید باید بپرسم کدهای موجود درست هستن یا نه.
   * گام‌ها باید یکی یکی با تأیید تو انجام بشن.
   * هر گام خروجی مشخص داره (مثلاً ساخته شدن فایل‌ها یا جواب درست از endpoint).

4. **وضعیت فعلی**

   * `/ping` → ok
   * `/query` → ok
   * `/reindex` → ok
   * `/upload` (txt) → ok
   * Dependencies دائمی و Dockerfile تمیز → ok
   * MVP برای txt کامل شده.

5. **گام‌های بعدی پیشنهادی**

   * **C)** Loaderهای PDF/DOCX/CSV → تغییر ingestion و upload.
   * **D)** اتصال LLM محلی (Qwen2-7B-Instruct q4 با llama.cpp) برای `/chat`.
   * **E)** UI سبک (Streamlit یا React).
   * متریک‌ها و لاگ‌های ingestion (اختیاری).

---
</div>

In [1]:
# Create a Persian PDF with embedded Vazirmatn using PyMuPDF 1.26.x (mac).
from pathlib import Path
import fitz  # PyMuPDF

root = Path.cwd()
fonts_dir = root / "data" / "fonts"
assert fonts_dir.exists(), f"fonts dir not found: {fonts_dir}"

# pick a Vazirmatn .ttf (prefer Regular if present)
cands = sorted(fonts_dir.glob("Vazirmatn*.ttf"))
assert cands, f"No Vazirmatn *.ttf under {fonts_dir}"
font_path = next((p for p in cands if "Regular" in p.name), cands[0])

out = root / "data" / "raw" / "sample_fa.pdf"
out.parent.mkdir(parents=True, exist_ok=True)

doc = fitz.open()
# PyMuPDF 1.26.x: private API
fontname = doc._insert_font(fontname="Vazirmatn", fontfile=str(font_path))

page = doc.new_page()
rect = fitz.Rect(50, 50, 550, 800)
text = (
    "این یک PDF تست فارسی است.\n"
    "هدف: بررسی استخراج متن با فونت embed شده (Vazirmatn).\n"
    "موضوعات: RAG، embeddings، FAISS، normalization.\n"
)

page.insert_textbox(rect, text, fontsize=14, fontname=fontname)
doc.save(str(out))
doc.close()

print("Wrote:", out)
print("Exists:", out.exists(), "Size:", out.stat().st_size if out.exists() else 0)


TypeError: Document._insert_font() got an unexpected keyword argument 'fontname'

In [2]:
pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.6 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pathlib import Path
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

root = Path.cwd()
fonts_dir = root / "data" / "fonts"
font_path = fonts_dir / "Vazirmatn-Regular.ttf"   # یا یکی از فایل‌های .ttf که داری

out = root / "data" / "raw" / "sample_fa.pdf"
out.parent.mkdir(parents=True, exist_ok=True)

# Register font
pdfmetrics.registerFont(TTFont("Vazirmatn", str(font_path)))

# Create PDF
c = canvas.Canvas(str(out))
c.setFont("Vazirmatn", 14)
text = (
    "این یک PDF تست فارسی است.\n"
    "هدف: بررسی استخراج متن با فونت Vazirmatn.\n"
    "موضوعات: RAG، embeddings، FAISS، normalization.\n"
)
c.drawString(50, 750, text.split("\n")[0])
c.drawString(50, 730, text.split("\n")[1])
c.drawString(50, 710, text.split("\n")[2])
c.save()

print("OK ->", out)


OK -> /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw/sample_fa.pdf


In [9]:
# Make a correct Persian PDF with shaping (no OCR needed).
# Needs: pip install reportlab arabic-reshaper python-bidi

from pathlib import Path
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import arabic_reshaper
from bidi.algorithm import get_display

root = Path.cwd()
font_path = root/"data/fonts/Vazirmatn-Regular.ttf"  # یکی از .ttfهایی که داری
assert font_path.exists(), f"Font not found: {font_path}"

out = root/"data/raw/sample_fa_ok.pdf"
out.parent.mkdir(parents=True, exist_ok=True)

pdfmetrics.registerFont(TTFont("Vazirmatn", str(font_path)))
c = canvas.Canvas(str(out))
c.setFont("Vazirmatn", 14)

lines = [
    "این یک PDF تست فارسی است.",
    "هدف: بررسی استخراج متن با فونت Vazirmatn.",
    "موضوعات: RAG، embeddings، FAISS، normalization."
]

y = 750
for ln in lines:
    shaped = arabic_reshaper.reshape(ln)
    visual = get_display(shaped)          # bidi
    c.drawRightString(550, y, visual)     # راست‌چین
    y -= 20

c.save()
print("Wrote:", out, "Exists:", out.exists())


Wrote: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw/sample_fa_ok.pdf Exists: True


In [8]:
pip install python-bidi

Note: you may need to restart the kernel to use updated packages.


<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
حله امیر ✌️ کوتاه و دقیق:

## ✅ کارهای انجام‌شده

* Docker dev پایدار (run با `rag-local` + دو ترمینال).
* API کامل: `/ping`، `/query`، `/reindex`، `/upload`.
* پشتیبانی فرمت‌ها: `txt / pdf / docx / csv`.
* ماژول‌ها:

  * `src/loaders.py` (PDF/DOCX/CSV/TXT) – PDF: اول **PyMuPDF**، در صورت نیاز OCR.
  * `src/chunking.py` (چانکینگ هوشمند بر اساس فرمت/طول متن).
* `main.py`: `/upload` ارتقا یافت (فرمت‌های جدید + `ocr`).
* نمونه‌فایل‌ها ساخته شد (`sample_en.pdf`, `sample_fa_ok.pdf`, …).
* PDF فارسی سالم: ساخت با **ReportLab + arabic-reshaper + python-bidi** → استخراج متن OK.
* دیباگ پردازش PDF (پرینت ساده) و علت کندی با `reindex=true` مشخص شد.

## 📁 فایل‌هایی که اضافه/تغییر شد

* اضافه: `src/loaders.py`, `src/chunking.py`
* تغییر: `src/main.py` (ایمپورت‌ها + `/upload` + چانکینگ هوشمند)
* (اختیاری دست‌نخورده: `ingestion.py` ـ اگر بخوای CLI هم هوشمند شه، یک خط جایگزینی داریم)

## 🔜 گام‌های پیشنهادی بعدی (به‌ترتیب کم‌ریسک → پربازده)

1. **کیفیت و سرعت**

   * پیش‌فرض `/upload`: `reindex=false` (ری‌اینکس جداگانه).
   * ثبت پارامترهای چانک (size/overlap) داخل `manifest.jsonl`.
   * گارد بهتر برای PDF: مسیر **fitz → pypdf → OCR** با تایم‌اوت/try دقیق تا فریز نشه.
2. **بهبود متادیتا**

   * ذخیره `page_start/page_end` (وقتی شد) و `source_type` یکنواخت.
   * CSV: تشخیص delimiter خودکار.
3. **گام D**: اتصال LLM محلی (Qwen2-7B q4 با `llama.cpp`) و اضافه‌ی `/chat`.
4. **گام E**: UI سبک (Streamlit) برای upload/query سریع.
5. (اختیاری) متریک و لاگ ingestion.

## ⏱ چرا پاسخ‌ها گاهی دیر میاد؟

* وقتی `reindex=true` می‌زنی، کل ایندکس دوباره ساخته می‌شه (CPU روی مک ۲۰۱۵ → چند ثانیه).
* PDF با OCR یا PDFهایی که fitz روی‌شان دیر می‌خواند → کندی طبیعی.
* از طرف من: وقتی پاسخ خیلی طولانی/کد زیاد می‌نویسم، رندر/ارسال طول می‌کشه. از این به بعد **کوتاه‌تر** می‌نویسم مگر واقعاً لازم باشه.

### ترفندهای کاهش تأخیر

* برای تست ساده: `reindex=false` و ری‌اینکس را جدا بزن.
* OCR فقط وقتی PDF اسکن است.
* فایل‌های تست کوتاه نگه‌دار؛ کُد دیباگ را کم کنیم.



</div>

In [1]:
import subprocess

MODEL = "../../../data/models/qwen2-7b-instruct-q4_k_m.gguf"

def chat_once(prompt):
    cmd = [
        "./llama-cli",
        "--model", MODEL,
        "--prompt", prompt,
        "-n", "128",
        "-ngl", "0",
        "-c", "2048"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout

# تست دیالوگ
print(chat_once("سلام! حالت چطوره؟"))
print(chat_once("من یک سوال درباره RAG دارم."))


FileNotFoundError: [Errno 2] No such file or directory: './llama-cli'

<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
باشه—خیلی کوتاه و کاربردی.

# 📌 خلاصهٔ فوریِ وضعیت (قابل کپی برای چت جدید)

* **پلتفرم:** MacBook Pro 2015 (Intel), اجرای محلی + Docker.
* **RAG API (FastAPI):** `/ping`, `/query`, `/reindex`, `/upload` همگی OK.
* **Loaderها:** `txt / pdf / docx / csv` فعال؛ PDF با PyMuPDF → pypdf → (اختیاری) OCR.
* **Chunking:** هوشمند بر اساس نوع فایل؛ خروجی به `data/chunks/{chunks.jsonl, manifest.jsonl}`.
* **Embeddings/Index:** multilingual-e5-small (CPU), FAISS (IP با L2-normalize)، مسیر `data/index/`.
* **تست‌ها:** آپلود و ری‌ایندکس برای هر فرمت انجام شد؛ `/query` نتایج درست.
* **llama.cpp:** بیلد با CMake OK؛ باینری‌ها در `llama.cpp/build/bin/`.
* **مدل LLM:** `qwen2-7b-instruct-q4_k_m.gguf` در `data/models/`.
* **اجرای محلی LLM:** بهترین حالت پایدار فعلی در Jupyter = فلو ۴ سلولی با `pexpect` (سلول راه‌اندازی، تعریف `ask`, اجرای سؤال).
* **GPU/Metal:** روی Intel Iris Pro ناپایدار ⇒ اجرای **CPU-only** (`-ngl 0`) + `--no-warmup`.

## ⚙️ پارامترهای فعلی LLM (برای کار روان‌تر روی MBP 2015)

* context: `-c 768`
* max tokens: `-n 48` (برای سرعت؛ قابل افزایش در سؤالات کوتاه)
* threads: `-t 4` (با توجه به دما/لود می‌تونی 6–8 هم تست کنی)
* batch: `-b 128`
* GPU: `-ngl 0`
* warmup: `--no-warmup`



## ✅ کارهای انجام‌شده (چک‌لیست)

* ساختار پروژه، Dockerfile، requirements پایدار ✅
* ingestion+loaders+chunking ✅
* embeddings+faiss+query ✅
* آپلود چندفرمت + ری‌ایندکس خودکار ✅
* تست PDF فارسی (با فونت و reshaper) ✅
* بیلد و اجرای Qwen2-7B (CPU-only) ✅
* Jupyter flow پایدار با `pexpect` ✅

## 🔜 گام‌های بعدی پیشنهادی (خیلی کوتاه)

1. افزودن **Endpoint `/chat`** در FastAPI (one-shot زیر hood؛ بدون سشن—پایدار و ساده).
2. **Template سادهٔ Chat** (system prompt کوتاه، تاریخ، زبان).
3. **اتصال RAG→LLM**: `/query` → ساخت پیام‌های زمینه → پاسخ LLM.
4. (اختیاری) **/rerank** با cosine یا MMR برای کیفیت بهتر.
5. (اختیاری) **UI سبک** بعد از پایدارشدن `/chat`.

اگر موافقی، از **گام ۱** شروع کنیم. طبق روال خودت قبل از هر کد:

* جایگزین شود یا فایل جدید؟ (پیشنهاد: افزودن `/chat` به همین `main.py`)
* اجرا کجا؟ (هاست یا کانتینر؟ پیشنهاد: فعلاً هاست برای تست سریع)
* چند ترمینال؟ (یکی کافی‌ست؛ ترمینال دوم فقط برای curl تست)

</div>